<a href="https://colab.research.google.com/github/nightitachi/NLP_Project-2024_US_Election_Sentiment_on_X/blob/main/2024_US_Election_Sentiment_on_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opendatasets

In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/emirhanai/2024-u-s-election-sentiment-on-x')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alibelhrak 
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/emirhanai/2024-u-s-election-sentiment-on-x


100%|██████████| 14.5k/14.5k [00:00<00:00, 17.0MB/s]

# Importing Libraries

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# Loading Dataset

In [4]:
df_train = pd.read_csv('/content/2024-u-s-election-sentiment-on-x/train.csv')
df_val = pd.read_csv('/content/2024-u-s-election-sentiment-on-x/val.csv')
df_test = pd.read_csv('/content/2024-u-s-election-sentiment-on-x/test.csv')

In [5]:
df_train


,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,1,@user123,2024-11-03 08:45:00,Excited to see Kamala Harris leading the Democ...,Kamala Harris,Democratic Party,120,450,positive
1,2,@politicsFan,2024-11-03 09:15:23,Donald Trump's policies are the best for our e...,Donald Trump,Republican Party,85,300,positive
2,3,@greenAdvocate,2024-11-03 10:05:45,Jill Stein's environmental plans are exactly w...,Jill Stein,Green Party,60,200,positive
3,4,@indieVoice,2024-11-03 11:20:10,Robert Kennedy offers a fresh perspective outs...,Robert Kennedy,Independent,40,150,neutral
4,5,@libertyLover,2024-11-03 12:35:55,Chase Oliver's libertarian stance promotes tru...,Chase Oliver,Libertarian Party,30,120,positive
...,...,...,...,...,...,...,...,...,...
495,496,@user1067,2024-02-10 08:50:30,Kamala Harris is a symbol of progressive leade...,Kamala Harris,Democratic Party,95,360,positive
496,497,@econTalk7,2024-02-10 09:15:55,Trump's economic strategies are showing mixed ...,Donald Trump,Republican Party,100,440,neutral
497,498,@greenFutureNow7,2024-02-10 10:05:30,Jill Stein's solar projects are leading the wa...,Jill Stein,Green Party,75,240,positive
498,499,@indiePerspective7,2024-02-10 11:35:25,Robert Kennedy offers pragmatic solutions outs...,Robert Kennedy,Independent,18,85,neutral


In [6]:
df_train['candidate'].value_counts()

,count
candidate,
Kamala Harris,100
Donald Trump,100
Jill Stein,100
Robert Kennedy,100
Chase Oliver,100


In [7]:
df_train['sentiment'].value_counts()

,count
sentiment,
positive,328
neutral,126
negative,45
positive,1


# Data Preparation

In [8]:
df_train_text = df_train['tweet_text']
df_valid_text = df_val['tweet_text']
df_test_text = df_test['tweet_text']

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


tokenizer = Tokenizer()
tokenizer.fit_on_text(df_train_text)
trained_tweets_sequences = tokenizer.fit_on_sequences(df_train_text)
trained_padded_sequences = pad_sequences(trained_tweets_sequences , maxlen = 50 , padding='POST')


In [10]:
train_input = np.array(train_pad_sequences)
val_input = np.array(valid_pad_sequences)
test_input = np.array(test_pad_sequences)

train_label = np.array(train_onehot_encoder)
val_label = np.array(valid_onehot_encoder)
test_label = np.array(test_onehot_encoder)

NameError: name 'train_pad_sequences' is not defined

# Creating The Model

In [ ]:
vocab_size = len(word_index)+1
embedding_dim = 100

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])


model.compile(optimizer = 'Adam' , loss='categorical_crossentropy' , metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_input, train_label, epochs=64, batch_size=32, validation_data=(valid_input, valid_label))

In [ ]:
print("Train input shape:", train_input.shape)
print("Train label shape:", train_label.shape)
print("Validation input shape:", valid_input.shape)
print("Validation label shape:", valid_label.shape)
